## 文檔轉換音檔（Text to Speech）
使用 Python 的 gTTS（Google Text-to-Speech）套件

將一段英文文字轉換成語音（Speech），並儲存成音訊檔案（預設為 output_audio.wav）

In [ ]:
from gtts import gTTS

def text_to_speech(text, filename="output_audio.wav"):
    tts = gTTS(text=text, lang='en')
    tts.save(filename)
    print(f"Audio saved as {filename}")
    return filename

# 將輸入文字轉為語音文件
text = "The company's performance this quarter showed strong growth in revenue and profit margins, driven by increased sales and efficient cost management. Key product launches and innovations contributed to market share growth, while investments in sustainability improved both reputation and appeal to eco-conscious consumers. Employee efforts fostered a collaborative culture, which supported overall success. These results exceeded expectations, positioning the company well for future expansion into new markets."
audio_file = text_to_speech(text)

## 音訊特徵擷取（Extract Audio Features）

這段程式碼的功能是：**從音訊檔案中萃取聲音特徵（Audio Features）**，主要使用 `librosa` 套件，這是 Python 中專門用來處理音訊訊號的工具。

### 處理流程：
🔊 載入音訊 → 🎛️ 萃取特徵 → 🧮 組成特徵向量 → 📋 輸出特徵字典

### 📌 擷取的音訊特徵包括：

#### 🔹 Zero Crossing Rate（零交叉率，ZCR）
- 表示音訊信號變號的頻率（由正變負或反之）。
- 與聲音的粗糙度、破裂感、噪音有關。
  
#### 🔹 Spectral Centroid（頻譜質心）
- 頻譜的「重心」，描述能量分布的平均位置。
- 可反映聲音的**亮度**或**尖銳程度**。

#### 🔹 Spectral Roll-off（頻譜滾降）
- 表示頻譜中某比例（如 85%）能量以下的最高頻率。
- 用來判斷聲音的**清晰度**與**高頻能量分佈**。

#### 🔹 MFCCs（梅爾頻率倒譜係數）
- 常用於語音與音樂辨識。
- 模擬人類聽覺系統，用來描述聲音的**音色特性**。

這些特徵將會組成一個向量，提供後續分析（如情緒預測、語音辨識）使用。


In [ ]:
import librosa
import numpy as np

def extract_audio_features_with_librosa(audio_file):
    # 加載音頻文件
    y, sr = librosa.load(audio_file, mono=True)
    
    # 計算特徵
    zcr = np.mean(librosa.feature.zero_crossing_rate(y)[0])
    spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr)[0])  
    spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr)[0])  
    mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr), axis=1) 

    # 組合特徵
    features = {
        "zcr": zcr,
        "spectral_centroid": spectral_centroid,
        "spectral_rolloff": spectral_rolloff,
    }
    features.update({f"mfcc_{i+1}": mfcc for i, mfcc in enumerate(mfccs)})
    return features

audio_file = "/Users/andrewhsu/Documents/FinTech_emotion/output_audio.wav"  
audio_features = extract_audio_features_with_librosa(audio_file)
print("Extracted Audio Features:")
print(audio_features)

Extracted Audio Features:
{'zcr': 0.1454309971389426, 'spectral_centroid': 2382.1094134918917, 'spectral_rolloff': 4127.072314736461, 'mfcc_1': -301.59106, 'mfcc_2': 86.60667, 'mfcc_3': 2.6235971, 'mfcc_4': 9.456604, 'mfcc_5': 2.0202246, 'mfcc_6': -8.451384, 'mfcc_7': -9.348526, 'mfcc_8': -12.273613, 'mfcc_9': -9.626634, 'mfcc_10': -11.328415, 'mfcc_11': -11.634263, 'mfcc_12': -0.62094, 'mfcc_13': -6.8831778, 'mfcc_14': 2.655982, 'mfcc_15': -4.8050957, 'mfcc_16': 1.0596268, 'mfcc_17': 2.6891272, 'mfcc_18': -1.468976, 'mfcc_19': -2.6905656, 'mfcc_20': -5.8471866}


# Pleasure-Arousal Model
### 🎧 語音特徵映射至情緒空間模型說明

本模型旨在將語音音訊所提取的特徵數據，映射至一個二維的情緒空間（Pleasure–Arousal Model）中，以量化語音表現的情緒傾向。

情緒空間使用以下兩個維度：
- **Pleasure（愉悅度）**：代表情緒的正負程度（例如：愉快 vs. 不悅）
- **Arousal（喚醒度）**：代表情緒的活躍程度（例如：激動 vs. 平靜）

流程如：
1. 使用 `librosa` 擷取音訊的特徵，包括：
   - Zero Crossing Rate
   - Spectral Centroid
   - Spectral Rolloff
   - MFCCs（梅爾頻率倒譜係數）
2. 將所有特徵使用 `MinMaxScaler` 標準化至 \[-1, 1\] 區間。
3. 建立兩個線性模型（LinearRegression）分別預測愉悅度與喚醒度。
4. 目前模型權重為模擬用（隨機產生），可替換為訓練模型進行實際應用。

此模型可應用於語音情緒分析、語音介面回饋、情緒輔助工具等領域，幫助系統理解語者當下的情緒狀態。


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import numpy as np

# 調整情緒模型的映射函數
def map_to_emotion_space(features):
    # 標準化特徵值
    scaler = MinMaxScaler(feature_range=(-1, 1))
    feature_values = np.array(list(features.values())).reshape(1, -1)
    normalized_features = scaler.fit_transform(feature_values).reshape(-1)
    
    # 線性模型參數
    pleasure_model = LinearRegression()
    arousal_model = LinearRegression()
    
    # 設置合理的模型参数
    np.random.seed(42)  # 固定随机种子，保证结果一致
    pleasure_model.coef_ = np.random.uniform(-1, 1, len(normalized_features))  # 调整范围
    pleasure_model.intercept_ = 0.5  # 设置偏置
    arousal_model.coef_ = np.random.uniform(-1, 1, len(normalized_features))
    arousal_model.intercept_ = -0.5  # 设置偏置
    
    # 計算愉悦度和唤醒度
    pleasure = pleasure_model.predict([normalized_features])[0]
    arousal = arousal_model.predict([normalized_features])[0]
    
    return {"pleasure": pleasure, "arousal": arousal}

## 🎯 視覺化情緒空間（Emotion Space Visualization）

此段程式碼會將前面模型輸出的情緒向量（Pleasure, Arousal）**繪製成一張二維圖像**，以呈現在 **Circumplex Model of Affect（情緒環模型）** 中的位置。

### 🗺️ 模型說明：
- **X 軸（Pleasure）**：代表正面 vs. 負面的情緒（如：開心 😄 vs. 憤怒 😠）
- **Y 軸（Arousal）**：代表高活躍 vs. 低活躍（如：興奮 🤩 vs. 放鬆 😌）

這張圖可用於理解語音的整體情緒傾向，例如：
- 右上：活躍且正面 → 興奮、愉悅
- 左下：低活躍且負面 → 沮喪、厭倦

### 📈 圖像功能：
- 紅點代表此語音在情緒空間中的位置。
- 藍色文字標註其對應的座標值。
- 使用 `matplotlib` 繪圖，直觀地呈現模型輸出結果。

這個圖非常適合做情緒分析結果的可視化，也能與多組語音資料比較彼此的情緒分佈。


In [ ]:
import matplotlib.pyplot as plt

# 绘图函数
def plot_emotion_space(emotion_space):
    pleasure = emotion_space["pleasure"]
    arousal = emotion_space["arousal"]
    
    # 绘制二维情绪空间
    plt.figure(figsize=(6, 6))
    plt.axhline(0, color='gray', linewidth=0.8)  # 横轴
    plt.axvline(0, color='gray', linewidth=0.8)  # 纵轴
    plt.scatter(pleasure, arousal, color='red', label="Emotion Point")
    plt.text(pleasure, arousal, f"({pleasure:.2f}, {arousal:.2f})", fontsize=12, color='blue', ha='center', va='bottom')
    plt.xlim(-1.5, 1.5)  # 限制 X 轴范围
    plt.ylim(-1.5, 1.5)  # 限制 Y 轴范围
    plt.xlabel("Pleasure")
    plt.ylabel("Arousal")
    plt.title("Emotion in Circumplex Model")
    plt.grid()
    plt.legend()
    plt.show()

# 执行修订流程
emotion_space = map_to_emotion_space(audio_features)
print("Emotion Space Mapping (Normalized):")
print(emotion_space)

plot_emotion_space(emotion_space)